**Test for Python Developer – API Data Extraction role**

1. Uses Instagrams profile directory for getting profile ids (https://www.instagram.com/directory/profiles/)

In [1]:

import requests
from bs4 import BeautifulSoup
import os
import time
import re
import pandas as pd  
import urllib
from PIL import Image

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException

**step 1 :** Get insta user links from https://www.instagram.com/directory/profiles/
give frive access to notebook

In [2]:
PROJECT_DIR = os.getcwd()
print(PROJECT_DIR)
DATA_DIR = os.path.join(PROJECT_DIR,'ScrappedData')
INSTA_DATA_DIR = os.path.join(DATA_DIR,'InstaDataFromInstaDirectory')

c:\Users\user\Desktop\data_scrape_test


In [3]:
if not os.path.exists(PROJECT_DIR):
  os.mkdir(PROJECT_DIR)

if not os.path.exists(DATA_DIR):
  os.mkdir(DATA_DIR)

if not os.path.exists(INSTA_DATA_DIR):
  os.mkdir(INSTA_DATA_DIR)

In [21]:
# !pip install webdriver-manager

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


In [4]:
custom_path=r'c:\Users\user\Desktop\data_scrape_test'

driver = webdriver.Chrome(ChromeDriverManager(path=custom_path).install())




====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_win32.zip
Driver has been saved in cache [c:\Users\user\Desktop\data_scrape_test\drivers\chromedriver\win32\93.0.4577.63]


In [143]:
!pip install insta-scrape
#xlsx
!pip install xlsxwriter==3.0.1 


You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: xlsxwriter
    Found existing installation: XlsxWriter 1.2.8
    Uninstalling XlsxWriter-1.2.8:
      Successfully uninstalled XlsxWriter-1.2.8


In [5]:
insta_profile_dir_url = 'https://www.instagram.com/directory/profiles'


In [28]:
from instascrape import *
import xlsxwriter
headers = {
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36 Edg/87.0.664.57",
    "cookie": "sessionid=33726898013%3AK7qCynVpWvzlot%3A8;"
}

In [9]:
def get_profile_url(heading_element):
  link = heading_element.find_parent('a')
  link_href = link.get('href')
  profile_link = link_href.split('=')[1]
  profile_link = profile_link.split('/&')[0]
  return profile_link

def get_image(url,name):
   urllib.request.urlretrieve(url, name)
   i = Image.open(name)
   i.thumbnail((100, 100), Image.LANCZOS)
   i.save(name)

def get_twitter_acc(username):
  search_query = 'https://www.google.com/search?q=site%3Atwitter.com+and+intitle:'+username
  data = requests.get(search_query)
  soup = BeautifulSoup(data.content, "html.parser")
  #extract headings
  headings = soup.find_all( 'h3' )
  #loop over headings of curr page 
  twitter_link = ' '
  twitter_id= ' '
  for heading in headings:
    if username in heading.getText().lower():
      twitter_id = re.findall('\(@.*?\)', heading.getText())
      if len(twitter_id) > 0:
        twitter_id= twitter_id[0].replace('(','').replace(')','')
        twitter_link = get_profile_url(heading).split('&')[0]
        return twitter_link
        
def get_youtube_acc(username):
  search_query = 'https://www.google.com/search?q='+username
  data = requests.get(search_query)
  soup = BeautifulSoup(data.content, "html.parser")
  #extract headings
  headings = soup.find_all( 'h3' )
  #loop over headings of curr page 
  youtube_link = ' '
  for heading in headings:
    parent_el = str(heading.parent)
    if 'youtube.com' in parent_el and 'watch' not in parent_el and username in parent_el:
      youtube_link = parent_el.split('=')[2].split('&')[0]
      return youtube_link


In [11]:
def scrape_insta_profile(insta_id):
  print("Scrapping details from id :"+str(insta_id))
  user = Profile(insta_id)
  user.scrape(headers=headers)
  follower_count = user.followers
  if(follower_count > 5000):
    following_count = user.following
    username = user.username
    full_name = user.full_name
    url = user.url
    verfified = str(user.is_verified)
    profile_pic_url = user.profile_pic_url
    category = user.category_enum
    num_posts = user.posts
    biography = user.json_dict['entry_data']['ProfilePage'][0]['graphql']['user']['biography']
    connected_fb_page = user.json_dict['entry_data']['ProfilePage'][0]['graphql']['user']['connected_fb_page']
    web_page = user.json_dict['entry_data']['ProfilePage'][0]['graphql']['user']['external_url']
    is_business_account = user.json_dict['entry_data']['ProfilePage'][0]['graphql']['user']['is_business_account']
    twitter_acc_link = get_twitter_acc(full_name) 
    youtube_acc = get_youtube_acc(full_name)
    if (is_business_account):
      acc_type = 'Business'
    else:
      acc_type = ' '
    get_image(profile_pic_url,'profile_pic.png')

    workbook = xlsxwriter.Workbook(str(INSTA_DATA_DIR)+'/insta_'+str(insta_id)+'.xlsx')
    worksheet = workbook.add_worksheet()
    worksheet.write('A1', 'Name')
    worksheet.write('B1', 'UserName')
    worksheet.write('C1', 'Age')
    worksheet.write('D1', 'Topics')
    worksheet.write('E1', 'Followers')
    worksheet.write('F1', 'Following')
    worksheet.write('G1', 'AVG Likes')
    worksheet.write('H1', 'Avg. Comments')
    worksheet.write('I1', 'Eng Rate')
    worksheet.write('J1', 'Total Posts')
    worksheet.write('K1', 'Email')
    worksheet.write('L1', 'Website')
    worksheet.write('M1', 'Bio')
    worksheet.write('N1', 'Gender')
    worksheet.write('O1', 'Location')
    worksheet.write('P1', 'Language')
    worksheet.write('Q1', 'Youtube')
    worksheet.write('R1', 'Twitter')
    worksheet.write('S1', 'TikTok')
    worksheet.write('T1', 'Facebook')
    worksheet.write('U1', 'Profile Picture url')
    worksheet.write('V1', 'Profile Picture')
    worksheet.write('W1', 'Verified')
    worksheet.write('X1', 'Account type')

    worksheet.set_row(1,50)
    worksheet.set_column_pixels(22,2,60)

    worksheet.write('A2', full_name)
    worksheet.write('B2', username)
    worksheet.write('C2', 'NA')
    worksheet.write('D2', 'NA')
    worksheet.write('E2', follower_count)
    worksheet.write('F2', following_count)
    worksheet.write('G2', ' ')
    worksheet.write('H2', ' ')
    worksheet.write('I2', ' ')
    worksheet.write('J2', num_posts)
    worksheet.write('K2', ' ')
    worksheet.write('L2', web_page)
    worksheet.write('M2', biography)
    worksheet.write('N2', ' ')
    worksheet.write('O2', ' ')
    worksheet.write('P2', ' ')
    worksheet.write('Q2', youtube_acc)
    worksheet.write('R2', twitter_acc_link)
    worksheet.write('S2', ' ')
    worksheet.write('T2', connected_fb_page)
    worksheet.write('U2', profile_pic_url)
    worksheet.insert_image('V2', 'profile_pic.png')
    worksheet.write('W2', verfified)
    worksheet.write('X2',acc_type )

    worksheet.set_column('A:A', 30)

    #save recent postsheading

    worksheet.write('A11', 'Posts')
    worksheet.write('B11', 'Description')
    worksheet.write('C11', 'Hashtags')
    worksheet.write('D11', 'Tags')
    worksheet.write('E11', 'Likes')
    worksheet.write('F11', 'Comments')
    worksheet.write('G11', 'Image/Video URL')
    worksheet.write('H11', 'Post URL')
    recent_posts = user.get_recent_posts()
    row = 12
    for post in recent_posts:
      post_url = post.url
      post_media_url = post.display_url
      get_image(post_media_url,str(row)+'.png')
      try:
        post_description = post.json_dict['edge_media_to_caption']['edges'][0]['node']['text']
      except:
        post_description = ' '
      hashtags = regex = re.findall("#(\w+)", post_description)
      hashtags = "#"+",#".join(hashtags)
      num_likes = post.likes
      num_comments = post.comments

      worksheet.set_row(row-1,80)
      worksheet.set_column_pixels(1,row,100)


      worksheet.insert_image('A'+str(row), str(row)+'.png')
      worksheet.write('B'+str(row), post_description)
      worksheet.write('C'+str(row), hashtags)
      worksheet.write('D'+str(row), ' ')
      worksheet.write('E'+str(row), num_likes)
      worksheet.write('F'+str(row), num_comments)
      worksheet.write('G'+str(row), post_media_url)
      worksheet.write('H'+str(row), post_url)
      row = row +1

    workbook.close()
      

In [21]:
def get_profile_url_and_id(driver):
    user_names = []
    profile_urls = []
    source = driver.page_source
    soup = BeautifulSoup(source, 'lxml')
    user_name_elements = driver.find_elements_by_class_name(name='_7kTyW')
    for el in user_name_elements:
        user_name = el.get_property('text')
        user_names.append(user_name)
        url = el.get_property('href')
        profile_urls.append(url)

    return user_names,profile_urls

In [22]:
#create a double loop for generating directory urls
timeout = 7
usernames_list = []
profile_url_list = []
max_ids = 11000

for index1 in range(0,100):
    if(len(usernames_list) > max_ids):
        break
    for index2 in range(0,10):
        if(len(usernames_list) > max_ids):
            break
        insta_dir_url = "{}/{}-{}".format(insta_profile_dir_url,index1,index2)
        print("Url : "+insta_dir_url)
        driver.get(insta_dir_url)
        try:
            element_present = EC.presence_of_element_located((By.CLASS_NAME, 'LGb3y'))
            WebDriverWait(driver, timeout).until(element_present)
            user_names,profile_urls = get_profile_url_and_id(driver)
            usernames_list.extend(user_names)
            profile_url_list.extend(profile_urls)
            if(len(usernames_list) > max_ids):
                break
        except TimeoutException:
            print ("Timed out waiting for page to load")

Url : https://www.instagram.com/directory/profiles/0-0
Url : https://www.instagram.com/directory/profiles/0-1
Url : https://www.instagram.com/directory/profiles/0-2
Url : https://www.instagram.com/directory/profiles/0-3
Url : https://www.instagram.com/directory/profiles/0-4
Url : https://www.instagram.com/directory/profiles/0-5
Url : https://www.instagram.com/directory/profiles/0-6
Url : https://www.instagram.com/directory/profiles/0-7
Url : https://www.instagram.com/directory/profiles/0-8
Url : https://www.instagram.com/directory/profiles/0-9
Url : https://www.instagram.com/directory/profiles/1-0
Url : https://www.instagram.com/directory/profiles/1-1
Url : https://www.instagram.com/directory/profiles/1-2
Url : https://www.instagram.com/directory/profiles/1-3
Url : https://www.instagram.com/directory/profiles/1-4
Url : https://www.instagram.com/directory/profiles/1-5
Url : https://www.instagram.com/directory/profiles/1-6
Url : https://www.instagram.com/directory/profiles/1-7
Url : http

KeyboardInterrupt: 

In [23]:
print("Total insta ids fetched = {}".format(len(usernames_list)+1))

Total insta ids fetched = 23101


In [24]:
#save as csv file

dict = {'insta_id': usernames_list, 'profile_urls': profile_url_list}  

df = pd.DataFrame(dict) 
    
# saving the dataframe 
df.to_csv(os.path.join(DATA_DIR,'insta_users_data.csv')) 

In [26]:
insta_user_details = pd.read_csv(os.path.join(DATA_DIR,'insta_users_data.csv'))
insta_user_details.head()

,Unnamed: 0,insta_id,profile_urls
0,0,tntsportsbr,https://www.instagram.com/tntsportsbr/?utm_sou...
1,1,orlabaddie,https://www.instagram.com/orlabaddie/?utm_sour...
2,2,manheim.eu,https://www.instagram.com/manheim.eu/?utm_sour...
3,3,hi_songjoongki,https://www.instagram.com/hi_songjoongki/?utm_...
4,4,onyourm__ark,https://www.instagram.com/onyourm__ark/?utm_so...


In [30]:
insta_ids = insta_user_details['insta_id']
index = 16

In [31]:

for insta_id in insta_ids[index:]:
    print("Fetching insta_id no. {}".format(index))
    try:
        scrape_insta_profile(insta_id)
    except:
        print("some error in {}".format(insta_id))
    index = index + 1
    

Fetching insta_id no. 16
Scrapping details from id :achocolate.frogg
Fetching insta_id no. 17
Scrapping details from id :master.baran725
some error in master.baran725
Fetching insta_id no. 18
Scrapping details from id :finishroid
Fetching insta_id no. 19
Scrapping details from id :muskansharma.5


C:\Users\user\AppData\Local\Programs\Python\Python37-32\lib\site-packages\instascrape\core\_static_scraper.py:136: MissingCookiesWarning: Request header does not contain cookies! It's recommended you pass at least a valid sessionid otherwise Instagram will likely redirect you to their login page.
  MissingCookiesWarning


Fetching insta_id no. 20
Scrapping details from id :battlegroundsmobilein_official
Fetching insta_id no. 21
Scrapping details from id :afgan__
Fetching insta_id no. 22
Scrapping details from id :thegymcosmo
Fetching insta_id no. 23
Scrapping details from id :losgrandes.gg
Fetching insta_id no. 24
Scrapping details from id :chrismcg
Fetching insta_id no. 25
Scrapping details from id :day.arad
Fetching insta_id no. 26
Scrapping details from id :firaroid
Fetching insta_id no. 27
Scrapping details from id :hasanzangeneheinalo
Fetching insta_id no. 28
Scrapping details from id :kisstoferr
Fetching insta_id no. 29
Scrapping details from id :the.unique_art
Fetching insta_id no. 30
Scrapping details from id :ffesportsbr
Fetching insta_id no. 31
Scrapping details from id :3i3tu
Fetching insta_id no. 32
Scrapping details from id :_imagine.records
Fetching insta_id no. 33
Scrapping details from id :b.henrique27
Fetching insta_id no. 34
Scrapping details from id :iraqait
Fetching insta_id no. 35
S